Step 1. Run below cell to import all package.

In [1]:
from qsee.compilation.qcompilation import QuantumCompilation
from qsee.compilation.qsp import QuantumStatePreparation
from qsee.core import ansatz, state, gradient, optimizer
from qsee.backend import constant, utilities
import matplotlib.pyplot as plt
import os
import base.object
import base.circuit
import numpy as np, qiskit
%load_ext autoreload
%autoreload 2

Step 2. Run below cell to make sure that your environment is ok or not

In [2]:
qsp = QuantumStatePreparation.prepare([1, 0, 0, 0])

Step 3. Coding!!!

In [ ]:
def test(num_qubits, t1,thetas,shots):
    """
        Return array time , W , P

        INPUT:
          t: time
          thetas: array parameter (length 2*(N-1))
          shots: number of measurement times

    """
    time=np.linspace(0,t1,shots)
    arrayW=[]
    arrayP=[]

    for t in time:
        w = base.object.E(num_qubits, t,thetas) - base.object.E(num_qubits, 0,thetas)
        if(t!=0):
            p = w/t
        else:
            p = 0
        arrayW.append(w)
        arrayP.append(p)

    return time,arrayW,arrayP
max_values = []
max_indices = []
t = 10
num_qubits = 2
fig, axs = plt.subplots(10, 10, figsize=(50, 50))

count=0
for i in range(10):
    for j in range(10):
        count=count+1
        thetas = np.random.rand(2*(num_qubits-1))* 2 * np.pi
        ltime, arrayW, arrayP = test(num_qubits, t, thetas, 100)
        index = i * 10 + j
        max_value = np.max(arrayP)
        max_index = np.argmax(arrayP)  
        axs[i, j].plot(ltime, arrayP, label=f"Power {count} ")
        axs[i, j].scatter(ltime[max_index], max_value, color='red', label=f"Max: {np.round(max_value, 2)}")
        axs[i, j].set_xlabel('Time')
        axs[i, j].set_ylabel('Value')
        axs[i, j].legend(prop={'size': 20})
        
        max_values.append(max_value)
        max_indices.append(ltime[max_index])
max_P = np.max(max_values)
max_Pindex = np.argmax(max_values)
plt.tight_layout()

directory = 'imageresult'
if not os.path.exists(directory):
    os.makedirs(directory)
plt.savefig(os.path.join(directory, f'2QUBIT_100IMAGE(1).pdf'), format='pdf')
plt.show()
print(f"Đồ thị {max_Pindex+1}: P_MAX = {max_P}, t = {max_indices[max_Pindex]}")